In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于批量预测的AutoML表格分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建表格分类模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。该数据集不需要任何特征工程。在本教程中使用的数据集版本存储在公共云存储桶中。训练后的模型可以预测三种Iris花的种类：山鸢尾(setosa)、维吉尼亚(virginica)或变色鸢尾(versicolor)。

### 目标

在本教程中，您将从Python脚本创建一个AutoML表格分类模型，然后使用Vertex客户端库进行批量预测。您还可以选择使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并即时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，并在准备好时将结果存储在Cloud Storage存储桶中。

### 成本

本教程使用 Google Cloud（GCP）的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本的* google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可以的话，请确保在 GPU 运行时中运行这个笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您使用何种笔记本环境，以下步骤均为必需步骤。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插值到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改“REGION”变量，该变量用于此笔记本的其余操作。 以下是支持 Vertex 的地区。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练 Vertex。并非所有区域都支持所有 Vertex 服务。有关每个区域的最新支持，请参阅 [Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行现场教程会话，您可能正在使用共享测试帐户或项目。为了避免在创建的资源上出现用户之间的名称冲突，您为每个实例会话创建一个时间戳，并附加到本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的帐户。

**否则**，按照以下步骤操作：

在 Cloud Console 中，转到 [创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在 **服务帐户名称** 字段中输入一个名称，然后点击 **创建**。

在 **将此服务帐户授予项目访问权限** 部分，点击角色下拉列表。在过滤框中键入“Vertex”，然后选择 **Vertex管理员**。在过滤框中键入“Storage Object管理员”，然后选择 **Storage Object管理员**。

点击创建。一个包含您密钥的 JSON 文件将下载到您的本地环境中。

在下面的单元格中，将服务帐户密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

本教程旨在使用位于公共云存储桶中的训练数据，并使用本地云存储桶进行批量预测。您也可以选择使用您自己存储在本地云存储桶中的训练数据。

在下面设置您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和端点服务终端点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 自动机器学习常量

设置特定于AutoML数据集和训练的常量：

- 数据集架构：告诉`数据集`资源服务数据集的类型是什么。
- 数据标记（注释）架构：告诉`数据集`资源服务数据如何被标记（注释）。
- 数据集训练架构：告诉`管道`资源服务用于训练模型的任务（例如，分类）。

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)
# Tabular Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tables_1.0.0.yaml"

硬件加速器

为预测设置硬件加速器（例如，GPU）。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器镜像，以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 分配给每个 VM 的 GPU 容器镜像，您需要指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 容器（Docker）镜像

对于AutoML批量预测，由Vertex预测服务提前确定服务二进制的容器镜像。更具体地说，该服务将根据您选择的硬件加速器为模型选择适当的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`以配置VM的计算资源，用于预测。
 - `machine type`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

#教程

现在您已经准备好开始创建您自己的AutoML表格分类模型。

## 设置客户端

Vertex客户端库运行为客户端/服务器模型。在你的端口（Python脚本）中，你将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请提前设置所有这些客户端。

- 数据集服务（Dataset Service）用于`Dataset`资源。
- 模型服务（Model Service）用于`Model`资源。
- 管道服务（Pipeline Service）用于训练。
- 作业服务（Job Service）用于批量预测和自定义训练。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，您的第一步是创建一个`数据集`资源实例。这一步与Vision、Video和Language不同。对于那些产品，在创建`数据集`资源之后，然后使用`import_data`方法单独导入数据。

对于表格数据，数据的导入要推迟到训练流程开始训练模型时。我们要做什么不同呢？首先，您不会调用`import_data`方法。相反，在创建数据集实例时，您将在`数据集`资源的元数据中指定CSV文件的Cloud Storage位置或数据表的BigQuery位置，其中包含您的表格数据。

#### Cloud Storage

`metadata = {"input_config": {"gcs_source": {"uri": [gcs_uri]}}}`

Cloud Storage路径的格式为：

    gs://[bucket_name]/[folder(s)/[file]

#### BigQuery

`metadata = {"input_config": {"bigquery_source": {"uri": [gcs_uri]}}}`

BigQuery路径的格式为：

    bq://[collection].[dataset].[table]

请注意，`uri`字段是一个列表，您可以在数据分布在文件中时输入多个CSV文件或BigQuery表。

### 数据准备

对于表格数据，Vertex的`Dataset`资源有一些要求。

- 必须是CSV文件或BigQuery查询。

CSV

对于表格分类，CSV文件有一些要求：

- 第一行必须是标题 — 注意这跟视觉、视频和语言不同，这些情况下不需要标题。
- 除了一个列是特征以外，其他所有列都是特征。
- 有一个列是标签，你将在随后创建训练管道时指定。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的鸢尾花数据集的一个版本，使用一个CSV索引文件。

首先要快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道标签列的标题名称，这保存为`label_column`，用于训练。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

## 数据集

现在，您的客户已经准备就绪，训练模型的第一步是创建一个托管数据集实例，然后将标记的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。该函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
   - `display_name`：您选择的易读名称。
   - `metadata_schema_uri`：数据集类型的模式。
   - `metadata`：表格数据的云存储或BigQuery位置。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
   - `parent`：您的`Database`，`Model`和`Endpoint`资源的Vertex位置根路径。
   - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的方式。虽然这一步通常很快，但当您在项目中第一次使用它时，由于预配，可能会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，方法是调用一个操作方法：

| 方法         | 描述                    |
| ----------- | ------------------------ |
| result()    | 等待操作完成并以JSON格式返回结果对象。      |
| running()   | 返回运行操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
| cancel()    | 取消操作（可能需要最多30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("iris-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

## 训练模型

现在使用您的Vertex `Dataset`资源训练一个AutoML表格分类模型。要训练模型，按照以下步骤进行：

1. 为`Dataset`资源创建一个Vertex训练管道。
2. 执行管道开始训练。

### 创建一个训练流程

您可能会问，我们用管道做什么？通常在工作（如训练）具有多个步骤时，我们通常按顺序执行：执行步骤A，执行步骤B，依此类推。通过将步骤放入管道中，我们可以获得以下好处：

1. 可以重复使用于后续训练任务。
2. 可以将其封装为容器并作为批处理任务运行。
3. 可以进行分发。
4. 所有步骤与相同的管道任务相关联，用于跟踪进度。

使用这个辅助函数`create_pipeline`，它接受以下参数：

- `pipeline_name`：管道任务的可读名称。
- `model_name`：模型的可读名称。
- `dataset`：Vertex完全限定的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练任务要求的字典。

辅助函数调用`Pipeline`客户端服务的方法`create_pipeline`，该方法接受以下参数：

- `parent`：您的`Dataset`，`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练任务的完整规范。

让我们现在更深入地了解构建`training_pipeline`规范所需的*最低要求*：

- `display_name`：管道任务的可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练任务要求的字典。
- `model_to_upload`：模型的可读名称。
- `input_data_config`：数据集规范。
  - `dataset_id`：只有 Vertex 数据集标识符（非完全限定） - 这是完全限定标识符的最后部分。
  - `fraction_split`：如果指定，则用于训练、测试和验证的数据集百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务需求

接下来，构建任务需求。与其他参数不同，需要一个Google protobuf Struct，而不是Python（类似JSON格式）字典的`task`字段。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段是：

- `prediction_type`：我们是在进行"分类"还是"回归"预测。
- `target_column`：要预测的CSV标题列名称（即标签）。
- `train_budget_milli_node_hours`：为训练模型的最大计费时间预算，其中1000 = 1小时。
- `disable_early_stopping`：是True还是False，以告诉AutoML是否使用其判断力提前停止训练或在整个预算内进行训练。
- `transformations`：指定每个特征列的特征工程。

对于`transformations`，列表必须有每列的条目。外键字段指示相应列的特征工程类型。在本教程中，将其设置为`"auto"`，告诉AutoML自动确定。

最后，通过调用`create_pipeline`辅助函数来创建流水线，该函数返回一个训练流水线对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "sepal_width"}},
    {"auto": {"column_name": "sepal_length"}},
    {"auto": {"column_name": "petal_length"}},
    {"auto": {"column_name": "petal_width"}},
]

In [ ]:
PIPE_NAME = "iris_pipe-" + TIMESTAMP
MODEL_NAME = "iris_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "prediction_type": Value(string_value="classification"),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "disable_early_stopping": Value(bool_value=False),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关培训管道的信息

现在仅获取此培训管道实例的管道信息。通过调用作业客户端服务的 `get_training_pipeline` 方法，辅助函数可以获取此作业的作业信息，参数如下：

- `name`：Vertex 完全合格的管道标识符。

当模型训练完成时，管道状态将为 `PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练上述模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过减去 `结束时间` 和 `开始时间` 来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的顶点模型资源标识符，即流水线服务分配给它的。您可以从返回的流水线实例中获取此标识符，即字段 `model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

## 评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（保留）数据，由管道服务用来评估模型。

###列出所有分片的评估

请使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：表示`Model`资源的Vertex完全限定模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），我们然后打印出评估中每个指标的所有键名称，并对于一个小型集合（`logLoss`和`auPrc`），您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批量预测的模型部署

现在部署您为批量预测创建的训练好的 Vertex `Model` 资源。这与部署用于按需预测的 `Model` 资源有所不同。

对于在线预测，您需要：

1. 创建一个 `Endpoint` 资源用于部署 `Model` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

3. 发送在线预测请求到 `Endpoint` 资源。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求配置资源。

3. 批量预测请求的结果将返回给调用者。

4. 作业服务将取消为批量预测请求配置的资源。

## 发送批量预测请求

现在对您部署的模型进行批量预测。

制作测试项

您将使用合成数据作为测试数据项。不必担心我们使用合成数据 - 我们只是想展示如何进行预测。

In [ ]:
HEADING = "petal_length,petal_width,sepal_length,sepal_width"
INSTANCE_1 = "1.4,1.3,5.1,2.8"
INSTANCE_2 = "1.5,1.2,4.7,2.4"

### 制作批量输入文件

现在制作一个批量输入文件，并将其存储在本地云存储桶中。与图像、视频和文本不同，表格的批量输入文件仅支持CSV格式。对于CSV文件，您需要做如下操作：

- 第一行是包含特征（字段）标题名称的标题行。
- 每个剩余行是一个单独的预测请求，包含对应的特征数值。

例如：

    "feature_1", "feature_2", ...
    value_1, value_2, ...

In [ ]:
import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(HEADING + "\n")
    f.write(str(INSTANCE_1) + "\n")
    f.write(str(INSTANCE_2) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例扩展

处理批量预测请求时，您有几种计算实例扩展的选择：

- 单个实例：批量预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数量。当模型首次部署到实例时，固定数量的计算实例将被配置，并且批量预测请求将均匀地分布在它们之间。

- 自动扩展：批量预测请求被分配到可伸缩数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时要配置的实例数量，并根据负载情况设置要取消配置的数量，将计算实例的最大（`MAX_NODES）数量设置为要配置的实例数量。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发起批量预测请求

现在您的两个测试项目的批次已经准备好了，让我们进行批量请求。使用以下参数调用这个辅助函数`create_batch_prediction_job`：

- `display_name`：用于预测任务的人类可读名称。
- `model_name`：`Model`资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件在 Cloud 存储中的路径 - 这是您之前创建的文件。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的 Cloud 存储路径。
- `parameters`：用于处理预测结果的附加过滤参数。

辅助函数调用作业客户端服务的`create_batch_prediction_job`方法，使用以下参数：

- `parent`：Dataset、Model 和 Pipeline 资源的 Vertex 根路径。
- `batch_prediction_job`：批量预测作业的规范。

现在让我们深入了解`batch_prediction_job`的规范：

- `display_name`：用于预测批作业的人类可读名称。
- `model`：`Model`资源的 Vertex 完全限定标识符。
- `dedicated_resources`：为批量预测作业提供的计算资源。
  - `machine_spec`：要配置的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只会分配 CPU。
  - `starting_replica_count`：最初配置的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`：可扩展的最大计算实例数量，您之前设置为变量 `MAX_NODES`。
- `model_parameters`：用于处理预测结果的附加过滤参数。*注意*，图像分割模型不支持额外参数。
- `input_config`：要预测实例的输入来源和格式类型。
 - `instances_format`：批量预测请求文件的格式：仅支持`csv`。
 - `gcs_source`：您的批量预测请求的一个或多个 Cloud 存储路径列表。
- `output_config`：预测结果的输出目的地和格式。
 - `prediction_format`：批量预测响应文件的格式：仅支持`csv`。
 - `gcs_destination`：预测结果的输出目标。

此调用是一个异步操作。您将从响应对象中打印一些选择字段，包括：

- `name`：分配给批量预测作业的 Vertex 完全限定标识符。
- `display_name`：用于预测批作业的人类可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：是否提供了与预测结果相关的 True/False 解释（可解释性）。
- `state`：预测作业的状态（挂起、运行等）。

由于此调用需要一段时间才能执行，您可能会得到`JobState.JOB_STATE_PENDING`作为`state`的状态。

In [ ]:
BATCH_MODEL = "iris_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "csv"
OUT_FORMAT = "csv"  # [csv]

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME, None
)

现在获取您创建的批处理预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用以下助手函数 `get_batch_prediction_job`，带有以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定标识符。

助手函数调用作业客户端服务的 `get_batch_prediction_job` 方法，带有以下参数：

- `name`：批量预测作业的 Vertex 完全限定标识符。在本教程中，您将向其传递批量预测作业的 Vertex 完全限定标识符 -- `batch_job_id`。

助手函数将返回存储预测结果的 Cloud Storage 路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将为`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在指定Cloud Storage路径上的预测结果。预测结果将以CSV格式呈现，这是您在创建批量预测任务时指定的，在名为`prediction`的子文件夹下，该文件夹中将包含名为`predictions*.csv`的文件。

现在显示（cat）内容。您会看到多行，每行代表一个预测结果。

对于每个预测结果：

- 前四个字段是您进行预测的值（特征）。
- 其余字段是每个预测结果的置信度值，介于0和1之间。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.csv

        ! gsutil cat $folder/prediction*.csv
        break
    time.sleep(60)

整理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME